In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import log_loss, make_scorer
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn import preprocessing

import warnings
warnings.filterwarnings('ignore')

In [187]:
df = pd.read_csv('data/updated_again_data.csv')

In [188]:
basic_stats = pd.DataFrame()
df.head()

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,...,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,R_Reach_cms_Updates,B_Reach_cms_Updates
0,Henry Cejudo,Marlon Moraes,Marc Goddard,2019-06-08,"Chicago, Illinois, USA",Red,True,Bantamweight,5,0.0,...,0.0,8.0,Orthodox,162.56,162.56,135.0,31.0,32.0,162.56,170.18
1,Valentina Shevchenko,Jessica Eye,Robert Madrigal,2019-06-08,"Chicago, Illinois, USA",Red,True,Women's Flyweight,5,0.0,...,0.0,5.0,Southpaw,165.10,167.64,125.0,32.0,31.0,167.64,167.64
2,Tony Ferguson,Donald Cerrone,Dan Miragliotta,2019-06-08,"Chicago, Illinois, USA",Red,False,Lightweight,3,0.0,...,1.0,14.0,Orthodox,180.34,193.04,155.0,36.0,35.0,193.04,185.42
3,Jimmie Rivera,Petr Yan,Kevin MacDonald,2019-06-08,"Chicago, Illinois, USA",Blue,False,Bantamweight,3,0.0,...,0.0,6.0,Orthodox,162.56,172.72,135.0,26.0,29.0,172.72,170.18
4,Tai Tuivasa,Blagoy Ivanov,Dan Miragliotta,2019-06-08,"Chicago, Illinois, USA",Blue,False,Heavyweight,3,0.0,...,0.0,3.0,Southpaw,187.96,190.50,264.0,32.0,26.0,190.50,185.42


In [189]:
cols = [
 'Winner',
 'B_age',
 'B_Height_cms',
 'B_Reach_cms_Updates',
 'B_Weight_lbs',
 'R_Height_cms',
 'R_Reach_cms_Updates',
 'R_Weight_lbs',
 'R_age',
 'no_of_rounds']

#removing stance and weight class for now cuz lazy


#  'weight_class',
#  'R_Stance',
    
#  'B_Stance',

# 'R_fighter',
#  'B_fighter',

In [190]:
basic_stats = df[cols]

In [191]:
basic_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5144 entries, 0 to 5143
Data columns (total 10 columns):
Winner                 5144 non-null object
B_age                  4972 non-null float64
B_Height_cms           5144 non-null float64
B_Reach_cms_Updates    5144 non-null float64
B_Weight_lbs           5144 non-null float64
R_Height_cms           5144 non-null float64
R_Reach_cms_Updates    5144 non-null float64
R_Weight_lbs           5144 non-null float64
R_age                  5080 non-null float64
no_of_rounds           5144 non-null int64
dtypes: float64(8), int64(1), object(1)
memory usage: 402.0+ KB


# Replacing all nulls with bullshit filler vals for ez-pz first model

In [194]:
basic_stats["B_age"] = basic_stats["B_age"].fillna(basic_stats["B_age"].mean())
basic_stats["R_age"] = basic_stats["R_age"].fillna(basic_stats["R_age"].mean())

# basic_stats["B_Stance"] = basic_stats["B_Stance"].fillna("Orthodox")

# basic_stats["R_Stance"] = basic_stats["R_Stance"].fillna("Orthodox")


/Users/matthewhwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/matthewhwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [186]:
basic_stats["B_age"]

0        31.000000
1        32.000000
2        36.000000
3        26.000000
4        32.000000
5        33.000000
6        32.000000
7        25.000000
8        31.000000
9        34.000000
10       28.000000
11       35.000000
12       33.000000
13       30.000000
14       27.000000
15       26.000000
16       29.000000
17       26.000000
18       27.000000
19       37.000000
20       39.000000
21       30.000000
22       23.000000
23       29.000000
24       24.000000
25       26.000000
26       30.000000
27       28.000000
28       31.000000
29       34.000000
           ...    
5116     29.171963
5117     26.000000
5118     29.171963
5119     29.171963
5120     34.000000
5121     30.000000
5122     29.171963
5123     29.171963
5124     24.000000
5125     29.171963
5126     29.171963
5127     29.171963
5128     29.171963
5129     29.171963
5130     29.171963
5131     29.171963
5132     29.171963
5133     29.171963
5134     29.171963
5135     18.000000
5136     34.000000
5137     29.

In [154]:
basic_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5144 entries, 0 to 5143
Data columns (total 10 columns):
Winner                 5144 non-null object
B_age                  5144 non-null float64
B_Height_cms           5144 non-null float64
B_Reach_cms_Updates    5144 non-null float64
B_Weight_lbs           5144 non-null float64
R_Height_cms           5144 non-null float64
R_Reach_cms_Updates    5144 non-null float64
R_Weight_lbs           5144 non-null float64
R_age                  5144 non-null float64
no_of_rounds           5144 non-null int64
dtypes: float64(8), int64(1), object(1)
memory usage: 402.0+ KB


In [155]:
basic_stats.head()

,Winner,B_age,B_Height_cms,B_Reach_cms_Updates,B_Weight_lbs,R_Height_cms,R_Reach_cms_Updates,R_Weight_lbs,R_age,no_of_rounds
0,Red,31.0,167.64,170.18,135.0,162.56,162.56,135.0,32.0,5
1,Red,32.0,167.64,167.64,125.0,165.10,167.64,125.0,31.0,5
2,Red,36.0,185.42,185.42,155.0,180.34,193.04,155.0,35.0,3
3,Blue,26.0,170.18,170.18,135.0,162.56,172.72,135.0,29.0,3
4,Blue,32.0,180.34,185.42,250.0,187.96,190.50,264.0,26.0,3


In [156]:
basic_stats["Winner"] = basic_stats["Winner"].apply(lambda x: True if x == "Red" else False)

/Users/matthewhwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [87]:
# one_hot = pd.get_dummies((basic_stats["R_Stance"])
# basic_stats = basic_stats.drop('R_stance',axis = 1)
# basic_stats = basic_stats.join(one_hot)

In [157]:
basic_stats.head()

,Winner,B_age,B_Height_cms,B_Reach_cms_Updates,B_Weight_lbs,R_Height_cms,R_Reach_cms_Updates,R_Weight_lbs,R_age,no_of_rounds
0,True,31.0,167.64,170.18,135.0,162.56,162.56,135.0,32.0,5
1,True,32.0,167.64,167.64,125.0,165.10,167.64,125.0,31.0,5
2,True,36.0,185.42,185.42,155.0,180.34,193.04,155.0,35.0,3
3,False,26.0,170.18,170.18,135.0,162.56,172.72,135.0,29.0,3
4,False,32.0,180.34,185.42,250.0,187.96,190.50,264.0,26.0,3


In [158]:
y = basic_stats["Winner"]

In [159]:
basic_stats = basic_stats.drop("Winner",axis = 1)

In [160]:
X = basic_stats

In [161]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 0)

In [171]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [172]:
accuracy = clf.score(X_test,y_test)
conf_dict = {'TT': 0, 'FF':0, 'TF':0, 'FT':0}
tot = len(y_test)
for i in range(tot):
    abbrev = str(clf.predict(X_test)[i])[0] + str(y_test.iloc[i])[0]
    conf_dict[abbrev] += 1
recall = conf_dict['TT']/(conf_dict['TT'] + conf_dict['FT'])
precision = conf_dict['TT']/(conf_dict['TT'] + conf_dict['TF'])
print('Accuracy : ',round(accuracy,2))
print('Recall (reality): ',round(recall,2))
print('Precision (prediction): ',round(precision,2))

Accuracy :  0.66
Recall (reality):  0.85
Precision (prediction):  0.72


In [173]:
a = clf.feature_importances_
b = X_test.columns
c = zip(a,b)
for each in c:
    print(each)

(0.16125968951657194, 'B_age')
(0.1176032141606884, 'B_Height_cms')
(0.13648112526372483, 'B_Reach_cms_Updates')
(0.07382647036207589, 'B_Weight_lbs')
(0.11500728460263357, 'R_Height_cms')
(0.13361835211556028, 'R_Reach_cms_Updates')
(0.07328867017413802, 'R_Weight_lbs')
(0.16163807473228403, 'R_age')
(0.027277119072323074, 'no_of_rounds')
